# Іспит (90 хв): Міні‑проєкт у Jupyter Notebook

- Курс: "Основи генеративного ШІ" (Комп'ютерна інженерія, 2 курс)
- Використовуйте ті самі інструменти/моделі, що в практичних (GitHub Models/Azure AI Inference).
- Назва файлу перед здачею: `Прізвище І.Б._varNN.ipynb` (змініть у назві цього файлу перед завантаженням).
- Після завершення — завантажити на Google‑диск, вказаний викладачем.

## Структура іспиту
- **[0]** Титул і варіант (ПІБ, група, варіант, модель, endpoint)
- **[1]** Підготовка середовища (5-10 хв)
- **[2]** Базова генерація (15-20 хв)
- **[2b]** Діалоговий режим чат-бота (10-15 хв)
- **[3]** Покращення промпту (20-25 хв)
- **[4]** Індивідуальна частина (25-30 хв)
- **[5]** Рефлексія (5-10 хв)

## Індивідуальні варіанти
Детальний список варіантів та інструкції див. у файлі `Іспит_мініпроєкт_GenAI_осінь_2025.md`

**Блоки варіантів:**
- **Блок A (1-10)**: Основи комп'ютерних систем
- **Блок B (11-20)**: Програмне забезпечення та мережі  
- **Блок C (21-30)**: Інтернет речей та embedded системи

**Стандартна структура кожного варіанту:** генеруйте 3+ приклади → валідуйте формат → порівняйте результати → зробіть висновки.

Вкажіть нижче свій ПІБ, групу, варіант, модель та endpoint.

## [0] Титул і варіант
- ПІБ: <вкажіть>
- Група: <вкажіть>
- Варіант: <NN>
- Модель: <наприклад gpt-4o-mini/phi-4/mistral>
- Endpoint: https://models.inference.ai.azure.com
- Час початку: <вкажіть>

## Важливо
- Перевірте свій варіант у файлі `Іспит_мініпроєкт_GenAI_осінь_2025.md`
- Стандартна структура кожного варіанту: генеруйте 3+ приклади → валідуйте формат → порівняйте результати → зробіть висновки
- Усі варіанти мають однаковий рівень складності та однакову структуру виконання

In [ ]:
%pip install -q pandas jsonschema python-dotenv azure-ai-inference

Note: you may need to restart the kernel to use updated packages.


In [23]:
# [1] Підготовка середовища: імпорти, .env, перевірка токена та клієнта
import os, time, json
import pandas 
from typing import Dict, Any, Tuple

# .env (як у практичних)
try:
    from dotenv import load_dotenv
    load_dotenv()
except Exception:
    pass

GITHUB_TOKEN = os.getenv('GITHUB_TOKEN')
ENDPOINT = os.getenv('AZUREAI_INFERENCE_ENDPOINT', 'https://models.inference.ai.azure.com')
MODEL = os.getenv('GENAI_MODEL', 'gpt-4o-mini')
assert GITHUB_TOKEN, 'GITHUB_TOKEN не знайдено. Додайте у .env'
print('✅ Токен завантажено:', bool(GITHUB_TOKEN))
print('➡️ Endpoint:', ENDPOINT)
print('➡️ Model:', MODEL)

# Підготовка клієнта: azure-ai-inference або fallback на requests (узгоджено з практичними)
client_mode = 'requests'
try:
    from azure.ai.inference import ChatCompletionsClient
    from azure.core.credentials import AzureKeyCredential
    _client = ChatCompletionsClient(endpoint=ENDPOINT, credential=AzureKeyCredential(GITHUB_TOKEN))
    client_mode = 'azure-ai-inference'
except Exception:
    import requests
    _client = None

print('➡️ Client mode:', client_mode)

def ask_llm(system: str, user: str, temperature: float = 0.7, max_tokens: int = 256) -> Tuple[str, Dict[str, Any], float]:
    start = time.time()
    messages = [
        {"role": "system", "content": system},
        {"role": "user",   "content": user},
    ]
    if client_mode == 'azure-ai-inference':
        # Виклик через SDK (див. ПР3/ПР4/ПР6/ПР7)
        resp = _client.complete(messages=messages, model=MODEL, temperature=temperature, max_tokens=max_tokens)
        text = resp.choices[0].message.content if hasattr(resp.choices[0].message, 'content') else resp.choices[0].message['content']
        usage = getattr(resp, 'usage', {}) or {}
    else:
        import requests
        url = ENDPOINT.rstrip('/') + '/chat/completions'
        headers = {
            'Authorization': f'Bearer {GITHUB_TOKEN}',
            'Content-Type': 'application/json'
        }
        payload = {
            'model': MODEL,
            'messages': messages,
            'temperature': temperature,
            'max_tokens': max_tokens
        }
        r = requests.post(url, headers=headers, json=payload, timeout=60)
        r.raise_for_status()
        data = r.json()
        text = data['choices'][0]['message']['content']
        usage = data.get('usage', {})
    latency = time.time() - start
    return text, usage, latency

# Швидкий sanity-check (1 короткий запит)
txt, usg, lat = ask_llm(
    system="Ви лаконічний помічник.",
    user="Скажи 'готово'.",
    temperature=0.0, max_tokens=16
)
print('LLM OK, latency:', round(lat, 2), 's')
print('Response:', txt)


✅ Токен завантажено: True
➡️ Endpoint: https://models.inference.ai.azure.com
➡️ Model: gpt-4o-mini
➡️ Client mode: azure-ai-inference
LLM OK, latency: 1.63 s
Response: Готово.


## [2] Базова генерація (15-20 хв)
Створіть базовий промпт для вашої теми (згідно з варіантом). Прогоніть серію запусків із різними `temperature`/`max_tokens` (мін. 3). Зафіксуйте latency/usage та коротко оцініть якість.

**Приклад для варіанту 1 (Алгоритми сортування):**
- Тема: порівняння алгоритмів сортування
- Базовий промпт: "Опиши 3 алгоритми сортування: bubble sort, quick sort, merge sort"
- Параметри: temperature=[0.2, 0.7, 0.9], max_tokens=[256, 256, 512]

**Вимоги:**
- Використовуйте тему вашого варіанту
- Зробіть мінімум 3 запити з різними параметрами
- Запишіть результати у `results_base`
- Оцініть якість відповідей
- Зробіть висновки про вплив параметрів на результат

In [24]:
# Базова генерація для вашого варіанту
# Замініть цей код відповідно до вашої теми

# Приклад для варіанту 1 (Алгоритми сортування)
system = 'Ви експерт з алгоритмів. Пояснюйте чітко і структуровано.'
user_base = '''Опиши 3 алгоритми сортування: bubble sort, quick sort, merge sort.
Для кожного вкажи: назва, короткий опис, часову складність, переваги, недоліки.'''

# Замініть на ваш варіант:
# system = 'Ви експерт з <ваша домена>. Пояснюйте чітко і структуровано.'
# user_base = '''<ваш базовий запит згідно з варіантом>'''

settings = [(0.2, 256), (0.7, 256), (0.9, 512)]
results_base = []
for t, mx in settings:
    text, usage, latency = ask_llm(system, user_base, temperature=t, max_tokens=mx)
    results_base.append({
        'temperature': t, 
        'max_tokens': mx, 
        'latency_s': latency, 
        'usage': usage, 
        'text': text[:400]
    })
    print(f"T={t}, Tokens={mx}: {text[:100]}...")

print("\n=== Результати базової генерації ===")
for i, res in enumerate(results_base):
    print(f"\n{i+1}. Temp={res['temperature']}, Latency={res['latency_s']:.2f}s")
    print(f"   Usage: {res['usage']}")
    print(f"   Text: {res['text'][:150]}...")

# Висновки про вплив параметрів на якість
print("\n=== Висновки про вплив параметрів ===")
print("Аналіз результатів:")
print(f"- Temperature 0.2: більш стабільні, передбачувані відповіді")
print(f"- Temperature 0.7: збалансовані відповіді з креативністю")
print(f"- Temperature 0.9: більш різноманітні, але менш передбачувані")
print(f"- Latency зростає зі збільшенням max_tokens")
print(f"- Висока температура дає кращу деталізацію, але нижчу стабільність")

# Порівняння якості
best_quality = 0
best_temp = None
for i, res in enumerate(results_base):
    # Проста оцінка якості за довжиною та змістом
    quality_score = len(res['text']) / (res['latency_s'] + 1)
    if quality_score > best_quality:
        best_quality = quality_score
        best_temp = res['temperature']

print(f"\nНайкраще співвідношення якості/швидкості: Temperature={best_temp}")
print("Рекомендації:")
print("- Для технічних завдань використовувати Temperature 0.2-0.4")
print("- Для креативних завдань - Temperature 0.7-0.9")
print("- Max_tokens підбирати залежно від необхідної деталізації")

results_base

T=0.2, Tokens=256: Ось опис трьох популярних алгоритмів сортування: bubble sort, quick sort та merge sort.

### 1. Bubb...
T=0.7, Tokens=256: Ось опис трьох популярних алгоритмів сортування: Bubble Sort, Quick Sort і Merge Sort.

### 1. Bubbl...
T=0.9, Tokens=512: Ось опис трьох популярних алгоритмів сортування:

### 1. Bubble Sort

- **Короткий опис**: Bubble So...

=== Результати базової генерації ===

1. Temp=0.2, Latency=4.25s
   Usage: {'completion_tokens': 256, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens': 78, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}, 'total_tokens': 334}
   Text: Ось опис трьох популярних алгоритмів сортування: bubble sort, quick sort та merge sort.

### 1. Bubble Sort

**Короткий опис:**
Bubble sort — це прост...

2. Temp=0.7, Latency=4.46s
   Usage: {'completion_tokens': 256, 'completion_tokens_details': {'accepted_prediction_to

[{'temperature': 0.2,
  'max_tokens': 256,
  'latency_s': 4.253131866455078,
  'usage': {'completion_tokens': 256, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens': 78, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}, 'total_tokens': 334},
  'text': 'Ось опис трьох популярних алгоритмів сортування: bubble sort, quick sort та merge sort.\n\n### 1. Bubble Sort\n\n**Короткий опис:**\nBubble sort — це простий алгоритм сортування, який працює шляхом повторного проходження через список, порівнюючи сусідні елементи і обмінюючи їх, якщо вони знаходяться в неправильному порядку. Процес повторюється, поки не буде виконано жодних обмінів, що означає, що списо'},
 {'temperature': 0.7,
  'max_tokens': 256,
  'latency_s': 4.458878993988037,
  'usage': {'completion_tokens': 256, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 

## [2b] Діалоговий режим чат-бота (5–10 хв)
Реалізуйте простий чат з історією повідомлень (`messages`). Використовуйте той самий клієнт/endpoint і функцію `ask_llm`.

Пояснення для студентів:
- Чат не запускається автоматично, щоб ноутбук не «зависав» у середовищах без stdin.
- Для запуску встановіть `RUN_CHAT = True` у кодовій клітинці нижче.
- Команди: `/exit` — вихід; `/reset` — очистити історію.
- Обмеження `MAX_TURNS` запобігає нескінченним діалогам.
- Рекомендовані параметри: `temperature≈0.3`, `max_tokens≈300` (можна змінювати).

In [25]:
# Діалоговий режим: безпечний чат-цикл з обмеженням ходів
import sys

messages = [{"role": "system", "content": "Ви помічник у діалоговому режимі. Відповідайте коротко і по суті."}]

def chat_once(user_text: str, temperature: float = 0.3, max_tokens: int = 300):
    global messages
    messages.append({"role": "user", "content": user_text})
    sys_prompt = next((m["content"] for m in reversed(messages) if m["role"] == "system"), "")
    text, usage, latency = ask_llm(sys_prompt, user_text, temperature=temperature, max_tokens=max_tokens)
    messages.append({"role": "assistant", "content": text})
    print(text)
    sys.stdout.flush()

RUN_CHAT = False  # встановіть True, щоб запустити чат
MAX_TURNS = 20    # безпечна межа кількості ходів

if RUN_CHAT:
    print("Чат-режим. Команди: /exit — вихід, /reset — очистити історію")
    turns = 0
    while turns < MAX_TURNS:
        try:
            user_in = input("you> ").strip()
        except EOFError:
            break
        if not user_in:
            continue
        if user_in == "/exit":
            print("Вихід з чат-режиму.")
            break
        if user_in == "/reset":
            messages = [{"role": "system", "content": "Ви помічник у діалоговому режимі. Відповідайте коротко і по суті."}]
            print("Історію очищено.")
            continue
        chat_once(user_in)
        turns += 1
    if turns >= MAX_TURNS:
        print("Досягнуто межі ходів (MAX_TURNS). Чат зупинено для безпеки.")

## [3] Покращення промпту (20-25 хв)
Додайте системний контекст, few-shot приклади, формат-контракт (JSON) та правила безпеки. Повторіть 1-2 запуски і порівняйте з [2].

**Приклад покращень для варіанту 1:**
- Системний контекст: детальна інструкція про JSON формат
- Few-shot: 1 приклад правильної структури
- Формат-контракт: сувора JSON схема
- Правила безпеки: перевірка синтаксису коду

**Вимоги:**
- Додайте мінімум 2 прийоми покращення промпту
- Вкажіть очікуваний формат відповіді (JSON)
- Додайте перевірку валідності результату
- Порівняйте якість з базовою генерацією
- Проведіть 2-3 запуски для стабільності результатів
- Проаналізуйте вплив кожного покращення

In [26]:
# Покращена генерація для вашого варіанту
# Замініть цей код відповідно до вашої теми

# Приклад для варіанту 1 (Алгоритми сортування)
system_improved = (
    'Ви експерт з алгоритмів. Генеруйте відповіді виключно у форматі JSON. '
    'Кожен алгоритм повинен мати поля: name, description, complexity, pros, cons. '
    'Будьте точними та структурованими. Перевіряйте синтаксис коду.'
)

# Few-shot приклад
fewshot = '''Приклад правильної відповіді:
{
  "algorithms": [
    {
      "name": "Example Sort",
      "description": "Опис алгоритму",
      "complexity": "O(n²)",
      "pros": ["просто реалізувати"],
      "cons": ["повільний на великих даних"]
    }
  ]
}

'''

user_improved = '''Опиши 3 алгоритми сортування: bubble sort, quick sort, merge sort.
Відповідь у форматі JSON з масивом "algorithms", де кожен об'єкт має поля:
name, description, complexity (найкращий/середній/найгірший випадки), pros, cons.'''

# Замініть на ваш варіант:
# system_improved = 'Ви експерт з <ваша домена>. <детальні інструкції>'
# fewshot = '''<приклад правильної відповіді у вашому форматі>'''
# user_improved = '''<ваш покращений запит з вказанням формату>'''

# Проведення 2-3 запусків для стабільності
improved_results = []
for attempt in range(3):
    print(f"\n=== Покращена генерація, спроба {attempt + 1} ===")
    text_i, usage_i, lat_i = ask_llm(system_improved, fewshot + user_improved, temperature=0.4, max_tokens=500)
    
    print('Покращена відповідь:')
    print(text_i[:300] + "..." if len(text_i) > 300 else text_i)
    print(f'Latency: {lat_i:.2f}s')
    print(f'Usage: {usage_i}')
    
    # Спроба розпарсити JSON
    try:
        import json
        data = json.loads(text_i)
        print('\n✅ JSON успішно розпарсено')
        print(f'Кількість алгоритмів: {len(data.get("algorithms", []))}')
        
        # Валідація структури
        algorithms = data.get('algorithms', [])
        valid_count = 0
        for alg in algorithms:
            required_fields = ['name', 'description', 'complexity', 'pros', 'cons']
            if all(field in alg for field in required_fields):
                valid_count += 1
        
        print(f'Алгоритмів з правильною структурою: {valid_count}/{len(algorithms)}')
        
        improved_results.append({
            'attempt': attempt + 1,
            'success': True,
            'data': data,
            'valid_algorithms': valid_count,
            'total_algorithms': len(algorithms),
            'usage': usage_i,
            'latency': lat_i,
            'text': text_i
        })
        
    except Exception as e:
        print(f'\n❌ Помилка парсингу JSON: {e}')
        improved_results.append({
            'attempt': attempt + 1,
            'success': False,
            'error': str(e),
            'usage': usage_i,
            'latency': lat_i,
            'text': text_i
        })

# Порівняння з базовою генерацією
print("\n" + "="*50)
print("ПОРІВНЯННЯ БАЗОВОЇ ТА ПОКРАЩЕНОЇ ГЕНЕРАЦІЇ")
print("="*50)

# Аналіз базової генерації
base_avg_latency = sum(r['latency_s'] for r in results_base) / len(results_base)
base_avg_tokens = sum(r['usage'].get('total_tokens', 0) for r in results_base) / len(results_base)

# Аналіз покращеної генерації
successful_improved = [r for r in improved_results if r['success']]
if successful_improved:
    improved_avg_latency = sum(r['latency'] for r in successful_improved) / len(successful_improved)
    improved_avg_tokens = sum(r['usage'].get('total_tokens', 0) for r in successful_improved) / len(successful_improved)
    success_rate = len(successful_improved) / len(improved_results) * 100
else:
    improved_avg_latency = 0
    improved_avg_tokens = 0
    success_rate = 0

print(f"Базова генерація:")
print(f"  Середній latency: {base_avg_latency:.2f}s")
print(f"  Середні токени: {base_avg_tokens:.0f}")
print(f"  Структурованість: низька (вільний текст)")

print(f"\nПокращена генерація:")
print(f"  Середній latency: {improved_avg_latency:.2f}s")
print(f"  Середні токени: {improved_avg_tokens:.0f}")
print(f"  Успішність JSON: {success_rate:.1f}%")
print(f"  Структурованість: висока (JSON)")

# Висновки про ефективність покращень
print(f"\nВисновки про ефективність покращень:")
if success_rate > 66:
    print("✅ Системний контекст та few-shot значно покращують структурованість")
    print("✅ JSON-контракт забезпечує стабільний формат")
else:
    print("⚠️ Потрібні додаткові налаштування для стабільності")

print("📈 Temperature 0.4 оптимальний для балансу стабільності та якості")
print("🎯 Few-shot приклади критично важливі для правильного формату")

{'latency_s': lat_i, 'usage': usage_i, 'preview': text_i[:500]}


=== Покращена генерація, спроба 1 ===
Покращена відповідь:
{
  "algorithms": [
    {
      "name": "Bubble Sort",
      "description": "Алгоритм, який повторно проходить через масив, порівнюючи сусідні елементи і обмінюючи їх місцями, якщо вони в неправильному порядку.",
      "complexity": {
        "best": "O(n)",
        "average": "O(n²)",
        "wors...
Latency: 6.88s
Usage: {'completion_tokens': 463, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens': 220, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}, 'total_tokens': 683}

✅ JSON успішно розпарсено
Кількість алгоритмів: 3
Алгоритмів з правильною структурою: 3/3

=== Покращена генерація, спроба 2 ===
Покращена відповідь:
{
  "algorithms": [
    {
      "name": "Bubble Sort",
      "description": "Алгоритм, що повторно проходить через список, порівнюючи сусідні елементи і обмінюючи їх місцями, якщо вони в н

{'latency_s': 7.804421901702881,
 'usage': {'completion_tokens': 472, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens': 220, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}, 'total_tokens': 692},
 'preview': '{\n  "algorithms": [\n    {\n      "name": "Bubble Sort",\n      "description": "Алгоритм сортування, який повторно проходить через список, порівнюючи сусідні пари елементів і обмінюючи їх, якщо вони в неправильному порядку.",\n      "complexity": {\n        "best": "O(n)",\n        "average": "O(n²)",\n        "worst": "O(n²)"\n      },\n      "pros": ["Простота реалізації", "Легко зрозуміти"],\n      "cons": ["Низька ефективність на великих даних", "Вимагає багато обчислювальних ресурсів"]\n    },\n    {\n '}

## [4] Індивідуальна частина (25-30 хв)

### Структура виконання:
1. **Генерація**: 3+ приклади з `temperature=0.4`, `max_tokens=600`
2. **Валідація**: JSON схема + `jsonschema`
3. **Аналіз**: pandas DataFrame + порівняння
4. **Висновки**: стабільність, якість, проблеми

### Індивідуальна частина для вашого варіанту (1-30)
### ЗАМІНІТЬ КОД НИЖЧЕ ВІДПОВІДНО ДО ВАШОГО ВАРІАНТУ

### Приклад для варіанту 1 (Комп'ютерні компоненти)
import jsonschema, pandas as pd

### Крок 1: Промпти
system_variant = 'Ви експерт з комп\'ютерного обладнання. Генеруйте у JSON форматі.'
user_variant = 'Опишіть 3 компоненти: CPU, RAM, SSD. Поля: name, function, speed, cost.'
fewshot_variant = '''Приклад:
{
  "components": [
    {"name": "Intel i7", "function": "Обробка даних", "speed": "3.5 GHz", "cost": "200-300 USD"}
  ]
}'''

### Крок 2: JSON схема
variant_schema = {
    "type": "object",
    "properties": {
        "components": {
            "type": "array", "minItems": 3,
            "items": {
                "type": "object",
                "properties": {
                    "name": {"type": "string"},
                    "function": {"type": "string"},
                    "speed": {"type": "string"},
                    "cost": {"type": "string"}
                },
                "required": ["name", "function", "speed", "cost"]
            }
        }
    },
    "required": ["components"]
}

### Крок 3: Генерація та валідація
variant_results = []
successful_attempts = 0

for attempt in range(3):
    print(f"\nСпроба {attempt + 1}")
    
    # Генерація
    text_v, usage_v, lat_v = ask_llm(
        system_variant, fewshot_variant + user_variant, 
        temperature=0.4, max_tokens=600
    )
    
    print(f'Latency: {lat_v:.2f}s')
    
    # Валідація
    try:
        data = json.loads(text_v)
        jsonschema.validate(data, variant_schema)
        
        components = data.get("components", [])
        valid_objects = sum(1 for comp in components 
                          if all(field in comp and comp[field] for field in ["name", "function", "speed", "cost"]))
        
        print(f'✅ JSON валідний: {valid_objects}/{len(components)} об\'єктів')
        
        successful_attempts += 1
        variant_results.append({
            'attempt': attempt + 1, 'success': True, 'data': data,
            'valid_objects': valid_objects, 'total_objects': len(components),
            'usage': usage_v, 'latency': lat_v
        })
        
    except Exception as e:
        print(f'❌ Помилка: {e}')
        variant_results.append({
            'attempt': attempt + 1, 'success': False, 'error': str(e),
            'usage': usage_v, 'latency': lat_v
        })

### Крок 4: Аналіз результатів
print("\n" + "="*50)
print("ПОРІВНЯЛЬНИЙ АНАЛІЗ")
print("="*50)

if successful_attempts > 0:
    all_components = []
    for result in variant_results:
        if result['success']:
            all_components.extend(result['data']['components'])
    
    df = pd.DataFrame(all_components)
    print("📊 Таблиця порівняння:")
    print(df[['name', 'speed', 'cost']].to_string(index=False))
    
    # Статистика
    print(f"\n📈 Статистика:")
    print(f"- Успішних спроб: {successful_attempts}/3 ({successful_attempts/3*100:.1f}%)")
    print(f"- Компонентів: {len(all_components)}")
    print(f"- Середній latency: {sum(r['latency'] for r in variant_results)/len(variant_results):.2f}s")

# Крок 5: Висновки
print(f"\n💡 ВИСНОВКИ:")
print(f"- Стабільність: {successful_attempts}/3 спроб успішних")
print(f"- JSON-контракт працює ефективно")
print(f"- Temperature 0.4 оптимальний")

if successful_attempts == 3:
    print("✅ Генерація стабільна")
elif successful_attempts >= 2:
    print("⚠️ Генерація частково стабільна")
else:
    print("❌ Генерація нестабільна")

### Фінальні результати
final_results = {
    "variant": 1,  # ЗАМІНІТЬ НА ВАШ ВАРІАНТ
    "attempts": 3, "successful": successful_attempts,
    "success_rate": successful_attempts / 3 * 100,
    "data": all_components if successful_attempts > 0 else []
}

print(f"\n📋 Результати:")
print(f"Варіант: {final_results['variant']}")
print(f"Успішність: {final_results['success_rate']:.1f}%")

### ЗАМІНІТЬ НА ВАШ ВАРІАНТ:
### 1. system_variant, user_variant, fewshot_variant
### 2. variant_schema для вашої структури  
### 3. Адапуйте аналіз під вашу тему
### 4. Замініть номер варіанту у final_results

In [27]:
# Адаптуйте цей код відповідно до вашого варіанту

import json
from typing import Dict, List, Any

# Спроба імпортувати pandas, якщо не встановлено - використовувати альтернативу
try:
    import pandas as pd
    HAS_PANDAS = True
except ImportError:
    HAS_PANDAS = False
    print("⚠️ Pandas не встановлено. Використовуємо альтернативний формат таблиці.")

def create_comparison_table(data: list, headers: list) -> str:
    """Створення текстової таблиці замість pandas DataFrame"""
    if not data:
        return "Немає даних для таблиці"
    
    # Розрахунок ширини колонок
    col_widths = [len(header) for header in headers]
    for row in data:
        for i, value in enumerate(row):
            if i < len(col_widths):
                str_value = str(value)
                col_widths[i] = max(col_widths[i], len(str_value))
    
    # Створення таблиці
    separator = "+" + "+".join("-" * (width + 2) for width in col_widths) + "+"
    table = [separator]
    
    # Заголовки
    header_row = "|" + "|".join(f" {header:<{col_widths[i]}} " for i, header in enumerate(headers)) + "|"
    table.append(header_row)
    table.append(separator)
    
    # Дані
    for row_data in data:
        row = "|" + "|".join(f" {str(row_data[i]):<{col_widths[i]}} " for i in range(len(headers))) + "|"
        table.append(row)
    
    table.append(separator)
    return "\n".join(table)

# Приклад для варіанту 1 (Алгоритми сортування) з валідацією JSON schema
def generate_algorithms_comparison() -> Dict[str, Any]:
    """Генерація порівняння алгоритмів сортування з повною валідацією"""
    
    # JSON схема для валідації
    schema = {
        "type": "object",
        "properties": {
            "algorithms": {
                "type": "array",
                "items": {
                    "type": "object",
                    "properties": {
                        "name": {"type": "string"},
                        "description": {"type": "string"},
                        "complexity": {
                            "type": "object",
                            "properties": {
                                "best": {"type": "string"},
                                "average": {"type": "string"},
                                "worst": {"type": "string"}
                            },
                            "required": ["best", "average", "worst"]
                        },
                        "pros": {"type": "array", "items": {"type": "string"}},
                        "cons": {"type": "array", "items": {"type": "string"}},
                        "code_example": {"type": "string"}
                    },
                    "required": ["name", "description", "complexity", "pros", "cons"]
                }
            }
        },
        "required": ["algorithms"]
    }
    
    system_prompt = (
        'Ви експерт з алгоритмів. Генеруйте відповіді виключно у форматі JSON. '
        'Для кожного алгоритму сортування надайте: name, description, complexity, pros, cons, code_example. '
        'Complexity в форматі {"best": "...", "average": "...", "worst": "..."} '
        'Код має бути синтаксично коректним Python.'
    )
    
    user_prompt = '''Згенеруй порівняння 3 алгоритмів сортування: bubble sort, quick sort, merge sort.
Відповідь у форматі JSON з полями:
{
  "algorithms": [
    {
      "name": "назва",
      "description": "короткий опис",
      "complexity": {"best": "...", "average": "...", "worst": "..."},
      "pros": ["перевага1", "перевага2"],
      "cons": ["недолік1", "недолік2"],
      "code_example": "код на Python"
    }
  ]
}'''
    
    # Проведення 2-3 запусків для стабільності
    attempts = []
    for attempt in range(3):
        print(f"\n=== Спроба {attempt + 1} ===")
        response, usage, latency = ask_llm(system_prompt, user_prompt, temperature=0.3, max_tokens=800)
        
        # Валідація JSON
        try:
            data = json.loads(response)
            
            # Валідація за схемою (спрощена, без jsonschema бібліотеки)
            validation_errors = []
            algorithms = data.get('algorithms', [])
            
            if len(algorithms) != 3:
                validation_errors.append(f"Очікувано 3 алгоритми, отримано {len(algorithms)}")
            
            for i, alg in enumerate(algorithms):
                required_fields = ['name', 'description', 'complexity', 'pros', 'cons']
                for field in required_fields:
                    if field not in alg:
                        validation_errors.append(f"Алгоритм {i}: відсутнє поле {field}")
                
                if 'complexity' in alg:
                    complexity_fields = ['best', 'average', 'worst']
                    for field in complexity_fields:
                        if field not in alg['complexity']:
                            validation_errors.append(f"Алгоритм {i}: відсутнє поле complexity.{field}")
            
            if validation_errors:
                print(f"❌ Помилки валідації: {validation_errors}")
                attempts.append({
                    'attempt': attempt + 1,
                    'success': False,
                    'errors': validation_errors,
                    'raw_response': response,
                    'usage': usage,
                    'latency': latency
                })
            else:
                print('✅ JSON валідний')
                
                # Створення таблиці порівняння
                comparison_data = []
                for alg in algorithms:
                    comparison_data.append([
                        alg.get('name', 'N/A'),
                        alg.get('complexity', {}).get('best', 'N/A'),
                        alg.get('complexity', {}).get('average', 'N/A'),
                        alg.get('complexity', {}).get('worst', 'N/A'),
                        ', '.join(alg.get('pros', [])),
                        ', '.join(alg.get('cons', []))
                    ])
                
                headers = ['Назва', 'Найкращий час', 'Середній час', 'Найгірший час', 'Переваги', 'Недоліки']
                
                print('\n📊 Таблиця порівняння:')
                if HAS_PANDAS:
                    df = pd.DataFrame(comparison_data, columns=headers)
                    print(df.to_string(index=False))
                else:
                    table = create_comparison_table(comparison_data, headers)
                    print(table)
                
                attempts.append({
                    'attempt': attempt + 1,
                    'success': True,
                    'data': data,
                    'comparison_table': comparison_data,
                    'headers': headers,
                    'usage': usage,
                    'latency': latency,
                    'raw_response': response
                })
                break  # Успішна спроба, виходимо з циклу
                
        except json.JSONDecodeError as e:
            print(f'❌ Помилка JSON: {e}')
            attempts.append({
                'attempt': attempt + 1,
                'success': False,
                'error': str(e),
                'raw_response': response,
                'usage': usage,
                'latency': latency
            })
    
    # Підсумкові висновки
    successful_attempts = [a for a in attempts if a['success']]
    if successful_attempts:
        best_attempt = successful_attempts[0]
        print(f"\n✅ Успішне виконання за {best_attempt['attempt']} спробу")
        
        # Аналіз результатів
        algorithms = best_attempt['data'].get('algorithms', [])
        print(f"\n📈 Аналіз продуктивності:")
        print(f"- Середній latency: {sum(a['latency'] for a in attempts) / len(attempts):.2f}s")
        print(f"- Загальні токени: {best_attempt['usage'].get('total_tokens', 0)}")
        
        # Порівняльний аналіз
        print(f"\n🔍 Порівняльний аналіз алгоритмів:")
        for alg in algorithms:
            complexity = alg.get('complexity', {})
            print(f"- {alg.get('name', 'N/A')}: {complexity.get('average', 'N/A')} середній час")
        
        return {
            'success': True,
            'attempts': attempts,
            'best_result': best_attempt,
            'conclusions': {
                'total_attempts': len(attempts),
                'successful_attempt': best_attempt['attempt'],
                'algorithms_analyzed': len(algorithms),
                'validation_passed': True
            }
        }
    else:
        print(f"\n❌ Всі спроби невдалі")
        return {
            'success': False,
            'attempts': attempts,
            'conclusions': {
                'total_attempts': len(attempts),
                'successful_attempt': None,
                'algorithms_analyzed': 0,
                'validation_passed': False
            }
        }

# Виконання для вашого варіанту
# result = generate_algorithms_comparison()
# result

# Замініть функцію вище для вашого варіанту:
# def generate_<your_topic>() -> Dict[str, Any]:
#     # ваш код тут
#     pass

# Розкоментуйте виклик вашої функції:
# result = generate_<your_topic>()
# result

## [5] Рефлексія (5-10 хв)
Коротко опишіть виконання завдання:

### Що спрацювало:
- Які промпт-інженерінгові техніки були найефективнішими?
- Які параметри (temperature, max_tokens) дали найкращі результати?
- Чи вдалося отримати структуровану відповідь у потрібному форматі?
- Які компоненти системи (клієнт, валідація, порівняння) працювали добре?

### Що не спрацювало:
- Які труднощі виникали з генерацією JSON?
- Чи доводилося робити кілька спроб для отримання коректного формату?
- Які обмеження моделі ви помітили?
- Які технічні проблеми виникали (API, парсинг, валідація)?

### Найбільший ефект дали:
- Детальний системний промпт з чіткими інструкціями
- Few-shot приклади для демонстрації формату
- Temperature = 0.3 для стабільності результатів
- Валідація JSON для перевірки коректності
- Більший час на виконання дозволив зробити більше спроб

### Обмеження та висновки:
- Складність отримання синтаксично коректного коду
- Необхідність кількох спроб для складних структур
- Важливість точного формулювання вимог до формату
- Роль валідації в забезпеченні якості результатів
- Вплив часу на якість та стабільність результатів

### Загальна оцінка:
- Чи вдалося виконати завдання повністю?
- Які б ви зробили покращення при повторному виконанні?
- Що ви навчилися про промпт-інженеринг для технічних завдань?
- Чи достатньо було 90 хвилин для якісного виконання?

# Завершення роботи

## 📋 Інструкції для здачі
1. **Перейменуйте файл**: `Прізвище І.Б._varNN.ipynb`
2. **Перевірте всі секції**: [0] → [1] → [2] → [2b] → [3] → [4] → [5]
3. **Запустіть всі клітинки**: Перевірте, що все працює коректно
4. **Очистіть вивід**: За бажанням очистьте довгі виводи для економії місця
5. **Збережіть файл**: Переконайтеся, що всі зміни збережено

## ✅ Checklist перед здачею
- [ ] ПІБ, група, варіант заповнені в секції [0]
- [ ] GITHUB_TOKEN працює (секція [1])
- [ ] Базова генерація виконана з 3+ параметрами (секція [2])
- [ ] Діалоговий режим готовий (секція [2b])
- [ ] Покращення промпту з few-shot та JSON (секція [3])
- [ ] Індивідуальна частина відповідно до варіанту (секція [4])
- [ ] Рефлексія з аналізом результатів (секція [5])
- [ ] Файл перейменовано правильно

## 🚀 Завантаження
- Завантажте файл на Google-диск, вказаний викладачем
- Будьте готові до короткої усної демонстрації (3-5 хв)

---
**Успіхів на іспиті!** 🎯